# Assignment 2 RL
### "Windy GridWorld Game"


Mina Rahmanian Shahri (20137470)

In [ ]:
import numpy as np
np.seterr(over='ignore')
import random
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from __future__ import print_function
from python_utils.import_ import import_global

## 4 Action Windy Gridworld Shown

Implementation of several algorithms (including: Sarsa, Q-learning, Sarsa(𝜆),and Watkins’s Q(𝜆) )  to solve this game's problem by 4 action (Up, Down, Right and Left).





In [ ]:

# wind gridworld height
WORLD_HEIGHT = 7

# wind gridworld width
WORLD_WIDTH = 10

# probability for exploration
EPSILON = 0.01

# Learning rate or step size
ALPHA = 0.1

# reward
REWARD = -1.0

# gamma
GAMMA = 0.9

# lambda
LAMBDA = 0.8

# wind strength for each column respectively
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

# possible 4 actions
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3

# value of the pairs state action
Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
START = [3, 0]
GOAL = [3, 7]

# eligibility trace
e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))


###############################  Functions  ####################################    

# state transitions
def step(currentState, currentAction):

    i, j = currentState
    
    # Up
    if currentAction == ACTION_UP:
        return [max(i - 1 - WIND[j], 0), j]
    # Down
    elif currentAction == ACTION_DOWN:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), j]
    # Left
    elif currentAction == ACTION_LEFT:
        return [max(i - WIND[j], 0), max(j - 1, 0)]
    # Right
    elif currentAction == ACTION_RIGHT:
        return [max(i - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
    else:
        assert False


# interaction in a Sarsa episode
def oneEpisode_s(Q):
  
    # track the total time steps in this episode
    time = 0

    # initialize state
    currentState = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  

    # keep going until get to the goal state
    while currentState != GOAL:   

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])
        

        # Sarsa update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])
                            
        currentState = newState
        currentAction = newAction
        time += 1

    return time

# interaction in a Sarsa_Lambda episode
def oneEpisode_S_lambda(Q):

    time = 0

    currentState = START

    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])

    # keep going until get to the goal state
    while currentState != GOAL:

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])

        # Sarsa Lambda update
        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], newAction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    e[i ,j, action] *= LAMBDA * GAMMA

        currentState = newState
        currentAction = newAction

        time += 1

    return time


# interaction in a Q-Learning episode
def oneEpisode_q(Q):
    
    time = 0
    
    currentState = START
    
    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        # selecting the new action among all possible action which gives the MAX Q
        newAction = np.argmax(Q[newState[0], newState[1], :])
        
        # Q-learning update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])

        currentState = newState

        time += 1

    return time


# interaction in a Q-Lambda episode
def oneEpisode_q_Lambda(Q):

    time = 0
    
    currentState = START

    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        newAction = np.argmax(Q[newState[0], newState[1], :])

        # Watkins’s Q_Lambda update          
        Bestaction = np.argmax(Q[newState[0], newState[1], actions])

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], Bestaction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    if newAction == Bestaction:
                          e[i ,j, action] *= LAMBDA * GAMMA
                    else:
                          e[i ,j, action] == 0      

  
        currentState = newState
        currentAction = newAction

        time += 1

    return time


# Preparing functions for plots

def sarsa():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    while ep < episodeLimit:
            #episodes.append(oneEpisode_s(Q))
            time = oneEpisode_s(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='royalblue')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 4 action SARSA, alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')


    
    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\t    Optimal policy SARSA is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')



def q_learning():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    while ep < episodeLimit:
            #episodes.append(oneEpisode_q(Q))
            time = oneEpisode_q(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='forestgreen')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 4 action Q-learning, alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')
    plt.show()


    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\t Optimal policy Q-learning is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')


def sarsa_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    while ep < episodeLimit:
            #episodes.append(oneEpisode_S_lambda(Q))
            time = oneEpisode_S_lambda(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='red')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 4 action SARSA ($\lambda$), alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')
    plt.show()

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\tOptimal policy SARSA_lambda is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column \n')


def q_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    while ep < episodeLimit:
            #episodes.append(oneEpisode_q_Lambda(Q))
            time = oneEpisode_q_Lambda(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='purple',)
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 4 action Watkins’s Q ($\lambda$), alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')
    plt.show()

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\tOptimal policy Watkins’s Q(λ) is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')


###############################################################################


if __name__ == '__main__':

    sarsa()
    q_learning()
    sarsa_Lambda()
    q_Lambda()



## Compare & Convergence --> prepare for figure

The convergence of the original implementation is compared with the implementation of all algorithms using a graph of Average Accumulated Steps vs. Episode.

In [ ]:

#######################   Convergence   ######################


WORLD_HEIGHT = 7
WORLD_WIDTH = 10

EPSILON = 0.01
ALPHA = 0.1
REWARD = -1.0
GAMMA = 0.9
LAMBDA = 0.8

# wind 
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3

Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
START = [3, 0]
GOAL = [3, 7]

# eligibility trace
e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

###############################  Functions  ####################################    

# state transitions
def step(currentState, currentAction):

    i, j = currentState
    
    # Up
    if currentAction == ACTION_UP:
        return [max(i - 1 - WIND[j], 0), j]
    # Down
    elif currentAction == ACTION_DOWN:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), j]
    # Left
    elif currentAction == ACTION_LEFT:
        return [max(i - WIND[j], 0), max(j - 1, 0)]
    # Right
    elif currentAction == ACTION_RIGHT:
        return [max(i - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
    else:
        assert False


# interaction in a Sarsa episode
def oneEpisode_s(Q):
  
    # track the total time steps in this episode
    time = 0

    # initialize state
    currentState = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  

    # keep going until get to the goal state
    while currentState != GOAL:   

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])
        

        # Sarsa update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])
                            
        currentState = newState
        currentAction = newAction
        time += 1

    return time

# interaction in a Sarsa_Lambda episode
def oneEpisode_S_lambda(Q):

    time = 0

    currentState = START

    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])

    # keep going until get to the goal state
    while currentState != GOAL:

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])

        # Sarsa Lambda update
        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], newAction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    e[i ,j, action] *= LAMBDA * GAMMA

        currentState = newState
        currentAction = newAction

        time += 1

    return time


# interaction in a Q-Learning episode
def oneEpisode_q(Q):
    
    time = 0
    
    currentState = START
    
    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        # selecting the new action among all possible action which gives the MAX Q
        newAction = np.argmax(Q[newState[0], newState[1], :])
        
        # Q-learning update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])

        currentState = newState

        time += 1

    return time


# interaction in a Q-Lambda episode
def oneEpisode_q_Lambda(Q):

    time = 0
    
    currentState = START

    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        newAction = np.argmax(Q[newState[0], newState[1], :])

        # Watkins’s Q_Lambda update          
        Bestaction = np.argmax(Q[newState[0], newState[1], actions])

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], Bestaction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    if newAction == Bestaction:
                          e[i ,j, action] *= LAMBDA * GAMMA
                    else:
                          e[i ,j, action] == 0      

  
        currentState = newState
        currentAction = newAction

        time += 1

    return time




# Preparing functions for plots
Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

episodeLimit = 1000
ep = 0
total_times = 0
x_s =[]
y_s =[]
while ep < episodeLimit:

          total_times += oneEpisode_s(Q)
          x_s.append(ep)
          y_s.append(total_times/(ep+1))
          ep += 1

# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')




Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

episodeLimit = 1000
ep = 0
total_times = 0
x_q=[]
y_q=[]
while ep < episodeLimit:

          total_times += oneEpisode_q(Q)
          x_q.append(ep)
          y_q.append(total_times/(ep+1))
          ep += 1

# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')




Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

episodeLimit = 1000
ep = 0
total_times = 0
x_sl=[]
y_sl=[]
while ep < episodeLimit:

          total_times += oneEpisode_S_lambda(Q)
          x_sl.append(ep)
          y_sl.append(total_times/(ep+1))
          ep += 1

# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')





Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

episodeLimit = 1000
ep = 0
total_times = 0
x_ql=[]
y_ql=[]
while ep < episodeLimit:

          total_times += oneEpisode_q_Lambda(Q)
          x_ql.append(ep)
          y_ql.append(total_times/(ep+1))
          ep += 1

# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')




plt.figure(figsize=(15,12))
plt.plot(x_s, y_s, color='darkblue', label='Sarsa')
plt.plot(x_q, y_q, color='seagreen', label='Q-Learning')
plt.plot(x_sl, y_sl, color='red', label='Sarsa(lambda)')
plt.plot(x_ql, y_ql, color='gold', label='Q(lambda)')
plt.legend(loc='best')
plt.xlabel('Episodes')
plt.ylabel('Average time steps')
plt.show()

# Compare all algorithms in 4 action




In [ ]:

#######################   Compare   ######################

WORLD_HEIGHT = 7
WORLD_WIDTH = 10
EPSILON = 0.01
ALPHA = 0.1
REWARD = -1.0
GAMMA = 0.9
LAMBDA = 0.8

# wind 
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3

Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
START = [3, 0]
GOAL = [3, 7]

# eligibility trace
e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))


###############################  Functions  ####################################    

# state transitions
def step(currentState, currentAction):

    i, j = currentState
    
    # Up
    if currentAction == ACTION_UP:
        return [max(i - 1 - WIND[j], 0), j]
    # Down
    elif currentAction == ACTION_DOWN:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), j]
    # Left
    elif currentAction == ACTION_LEFT:
        return [max(i - WIND[j], 0), max(j - 1, 0)]
    # Right
    elif currentAction == ACTION_RIGHT:
        return [max(i - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
    else:
        assert False


# interaction in a Sarsa episode
def oneEpisode_s(Q):
  
    # track the total time steps in this episode
    time = 0

    # initialize state
    currentState = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  

    # keep going until get to the goal state
    while currentState != GOAL:   

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])
        

        # Sarsa update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])
                            
        currentState = newState
        currentAction = newAction
        time += 1

    return time

# interaction in a Sarsa_Lambda episode
def oneEpisode_S_lambda(Q):

    time = 0

    currentState = START

    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])

    # keep going until get to the goal state
    while currentState != GOAL:

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])

        # Sarsa Lambda update
        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], newAction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    e[i ,j, action] *= LAMBDA * GAMMA

        currentState = newState
        currentAction = newAction

        time += 1

    return time


# interaction in a Q-Learning episode
def oneEpisode_q(Q):
    
    time = 0
    
    currentState = START
    
    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        # selecting the new action among all possible action which gives the MAX Q
        newAction = np.argmax(Q[newState[0], newState[1], :])
        
        # Q-learning update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])

        currentState = newState

        time += 1

    return time


# interaction in a Q-Lambda episode
def oneEpisode_q_Lambda(Q):

    time = 0
    
    currentState = START

    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        newAction = np.argmax(Q[newState[0], newState[1], :])

        # Watkins’s Q_Lambda update          
        Bestaction = np.argmax(Q[newState[0], newState[1], actions])

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], Bestaction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    if newAction == Bestaction:
                          e[i ,j, action] *= LAMBDA * GAMMA
                    else:
                          e[i ,j, action] == 0      

  
        currentState = newState
        currentAction = newAction

        time += 1

    return time





Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
# number of episodes
episodeLimit = 1000
ep = 0
episodes1 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_s(Q))
            time = oneEpisode_s(Q)
            episodes1.extend([ep] * time)
            ep += 1



    
# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')





Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
# number of episodes
episodeLimit = 1000
ep = 0
episodes2 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_q(Q))
            time = oneEpisode_q(Q)
            episodes2.extend([ep] * time)
            ep += 1




# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')




Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
# number of episodes
episodeLimit = 1000
ep = 0
episodes3 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_S_lambda(Q))
            time = oneEpisode_S_lambda(Q)
            episodes3.extend([ep] * time)
            ep += 1




# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')





Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
# number of episodes
episodeLimit = 1000
ep = 0
episodes4 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_q_Lambda(Q))
            time = oneEpisode_q_Lambda(Q)
            episodes4.extend([ep] * time)
            ep += 1



# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')



plt.figure(figsize=(10,7))
plt.plot(episodes1, color='darkblue',label="SARSA")
plt.plot(episodes3, color='seagreen',label="SARSA(λ)")
plt.plot(episodes2, color='red',label="Q Learning")
plt.plot(episodes4, color='gold',label="Q(λ)")
plt.grid(b=True, which='major', color='#666666', linestyle='--')
plt.title("All (Windy Gridworld) algorithms - 4 action, alpha={}, epsilon={}".format(ALPHA, EPSILON))
plt.legend(loc='lower right')
plt.xlabel('Time steps')
plt.ylabel('Episodes')
plt.show()



## Analyzed for 4 different values of $ \ Alpha $.

In [ ]:

def sarsa():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    episodeLimit = 1000
    ep = 0
    total_times1 = 0

    while ep < episodeLimit:

        total_times1  += oneEpisode_s(Q)
        x_sarsa_Alpha.append(ep)
        y_sarsa_Alpha.append(total_times1/(ep+1))
        ep = ep+1

    
    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if ALPHA == ALPHAS[0]:
        plt.plot(x_sarsa_Alpha, y_sarsa_Alpha, color='forestgreen', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[1]:
        plt.plot(x_sarsa_Alpha, y_sarsa_Alpha, color='royalblue', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[2]:
        plt.plot(x_sarsa_Alpha, y_sarsa_Alpha, color='red', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[3]:
        plt.plot(x_sarsa_Alpha, y_sarsa_Alpha, color='purple', label='Alpha='+str(ALPHA))


def q_learning():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    total_times2 = 0

    while ep < episodeLimit:

        total_times2  += oneEpisode_q(Q)
        x_q_Alpha.append(ep)
        y_q_Alpha.append(total_times2/(ep+1))
        ep = ep+1

    #episodes = np.add.accumulate(episodes)

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if ALPHA == ALPHAS[0]:
        plt.plot(x_q_Alpha, y_q_Alpha, color='forestgreen', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[1]:
        plt.plot(x_q_Alpha, y_q_Alpha, color='royalblue', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[2]:
        plt.plot(x_q_Alpha, y_q_Alpha, color='red', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[3]:
        plt.plot(x_q_Alpha, y_q_Alpha, color='purple', label='Alpha='+str(ALPHA))


def sarsa_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    total_times3 = 0

    while ep < episodeLimit:

        total_times3  += oneEpisode_S_lambda(Q)
        x_sarsaL_Alpha.append(ep)
        y_sarsaL_Alpha.append(total_times3/(ep+1))
        ep = ep+1

    #episodes = np.add.accumulate(episodes)

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if ALPHA == ALPHAS[0]:
        plt.plot(x_sarsaL_Alpha, y_sarsaL_Alpha, color='forestgreen', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[1]:
        plt.plot(x_sarsaL_Alpha, y_sarsaL_Alpha, color='royalblue', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[2]:
        plt.plot(x_sarsaL_Alpha, y_sarsaL_Alpha, color='red', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[3]:
        plt.plot(x_sarsaL_Alpha, y_sarsaL_Alpha, color='purple', label='Alpha='+str(ALPHA))


def q_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    total_times4 = 0

    while ep < episodeLimit:

        total_times4  += oneEpisode_q_Lambda(Q)
        x_ql_Alpha.append(ep)
        y_ql_Alpha.append(total_times4/(ep+1))
        ep = ep+1

    #episodes = np.add.accumulate(episodes)

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if ALPHA == ALPHAS[0]:
        plt.plot(x_ql_Alpha, y_ql_Alpha, color='forestgreen', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[1]:
        plt.plot(x_ql_Alpha, y_ql_Alpha, color='royalblue', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[2]:
        plt.plot(x_ql_Alpha, y_ql_Alpha, color='red', label='Alpha='+str(ALPHA))
    if ALPHA == ALPHAS[3]:
        plt.plot(x_ql_Alpha, y_ql_Alpha, color='purple', label='Alpha='+str(ALPHA))

   

if __name__ == '__main__':

    # Learning rate or step size
    ALPHAS = [0.1,0.2,0.4,0.5]
    plt.figure(figsize=(10,8))

    for ALPHA in ALPHAS:
       print('Calculating convergence for ALPHA in Sarsa= ', ALPHA)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_sarsa_Alpha = []
       y_sarsa_Alpha = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       sarsa()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $ALPHA$ in Sarsa')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show() 
##########################
  
    # Learning rate or step size
    ALPHAS = [0.1,0.2,0.4,0.5]
    plt.figure(figsize=(10,8))

    for ALPHA in ALPHAS:
       print('Calculating convergence for ALPHA in Q-Learning= ', ALPHA)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_q_Alpha = []
       y_q_Alpha = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       q_learning()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $ALPHA$ in Q-Learning')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show()

###########################

    # Learning rate or step size
    ALPHAS = [0.1,0.2,0.4,0.5]
    plt.figure(figsize=(10,8))

    for ALPHA in ALPHAS:
       print('Calculating convergence for ALPHA in Sarsa-Lambda= ', ALPHA)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_sarsaL_Alpha = []
       y_sarsaL_Alpha = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       sarsa_Lambda()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $ALPHA$ in Sarsa-Lambda')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show()

###########################

    # Learning rate or step size
    ALPHAS =[0.1,0.2,0.4,0.5]
    plt.figure(figsize=(10,8))

    for ALPHA in ALPHAS:
       print('Calculating convergence for ALPHA in Q-Lambda= ', ALPHA)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_ql_Alpha = []
       y_ql_Alpha = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       q_Lambda()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $ALPHA$ in Q-Lambda')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show()

 ## Analyzed for 4 different values of $ \ Epsilon $.

In [ ]:

def sarsa():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    episodeLimit = 1000
    ep = 0
    total_times1 = 0

    while ep < episodeLimit:

        total_times1  += oneEpisode_s(Q)
        x_sarsa_EPSILON.append(ep)
        y_sarsa_EPSILON.append(total_times1/(ep+1))
        ep = ep+1

    
    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if EPSILON == EPSILONs[0]:
        plt.plot(x_sarsa_EPSILON, y_sarsa_EPSILON, color='forestgreen', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[1]:
        plt.plot(x_sarsa_EPSILON, y_sarsa_EPSILON, color='royalblue', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[2]:
        plt.plot(x_sarsa_EPSILON, y_sarsa_EPSILON, color='red', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[3]:
        plt.plot(x_sarsa_EPSILON, y_sarsa_EPSILON, color='purple', label='EPSILON='+str(EPSILON))


def q_learning():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    total_times2 = 0

    while ep < episodeLimit:

        total_times2  += oneEpisode_q(Q)
        x_q_EPSILON.append(ep)
        y_q_EPSILON.append(total_times2/(ep+1))
        ep = ep+1

    #episodes = np.add.accumulate(episodes)

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if EPSILON == EPSILONs[0]:
        plt.plot(x_q_EPSILON, y_q_EPSILON, color='forestgreen', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[1]:
        plt.plot(x_q_EPSILON, y_q_EPSILON, color='royalblue', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[2]:
        plt.plot(x_q_EPSILON, y_q_EPSILON, color='red', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[3]:
        plt.plot(x_q_EPSILON, y_q_EPSILON, color='purple', label='EPSILON='+str(EPSILON))


def sarsa_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    total_times3 = 0

    while ep < episodeLimit:

        total_times3  += oneEpisode_S_lambda(Q)
        x_sarsaL_EPSILON.append(ep)
        y_sarsaL_EPSILON.append(total_times3/(ep+1))
        ep = ep+1

    #episodes = np.add.accumulate(episodes)

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if EPSILON == EPSILONs[0]:
        plt.plot(x_sarsaL_EPSILON, y_sarsaL_EPSILON, color='forestgreen', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[1]:
        plt.plot(x_sarsaL_EPSILON, y_sarsaL_EPSILON, color='royalblue', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[2]:
        plt.plot(x_sarsaL_EPSILON, y_sarsaL_EPSILON, color='red', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[3]:
        plt.plot(x_sarsaL_EPSILON, y_sarsaL_EPSILON, color='purple', label='EPSILON='+str(EPSILON))


def q_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    total_times4 = 0

    while ep < episodeLimit:

        total_times4  += oneEpisode_q_Lambda(Q)
        x_ql_EPSILON.append(ep)
        y_ql_EPSILON.append(total_times4/(ep+1))
        ep = ep+1

    #episodes = np.add.accumulate(episodes)

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
        optimalPolicy.append([])
        for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')

    if EPSILON == EPSILONs[0]:
        plt.plot(x_ql_EPSILON, y_ql_EPSILON, color='forestgreen', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[1]:
        plt.plot(x_ql_EPSILON, y_ql_EPSILON, color='royalblue', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[2]:
        plt.plot(x_ql_EPSILON, y_ql_EPSILON, color='red', label='EPSILON='+str(EPSILON))
    if EPSILON == EPSILONs[3]:
        plt.plot(x_ql_EPSILON, y_ql_EPSILON, color='purple', label='EPSILON='+str(EPSILON))

   

if __name__ == '__main__':

    # Learning rate or step size
    EPSILONs = [0.01,0.1,0.2,0.3]
    plt.figure(figsize=(10,8))

    for EPSILON in EPSILONs:
       print('Calculating convergence for EPSILON in Sarsa= ', EPSILON)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_sarsa_EPSILON = []
       y_sarsa_EPSILON = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       sarsa()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $EPSILON$ in Sarsa')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show() 
##########################
  
    # Learning rate or step size
    EPSILONs = [0.01,0.1,0.2,0.3]
    plt.figure(figsize=(10,8))

    for EPSILON in EPSILONs:
       print('Calculating convergence for EPSILON in Q-Learning= ', EPSILON)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_q_EPSILON = []
       y_q_EPSILON = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       q_learning()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $EPSILON$ in Q-Learning')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show()

###########################

    # Learning rate or step size
    EPSILONs = [0.01,0.1,0.2,0.3]
    plt.figure(figsize=(10,8))

    for EPSILON in EPSILONs:
       print('Calculating convergence for EPSILON in Sarsa-Lambda= ', EPSILON)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_sarsaL_EPSILON = []
       y_sarsaL_EPSILON = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       sarsa_Lambda()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $EPSILON$ in Sarsa-Lambda')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show()

###########################

    # Learning rate or step size
    EPSILONs = [0.01,0.1,0.2,0.3]
    plt.figure(figsize=(10,8))

    for EPSILON in EPSILONs:
       print('Calculating convergence for EPSILON in Q-Lambda= ', EPSILON)

       # Eligibility trace 
       e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))
    
    
       x_ql_EPSILON = []
       y_ql_EPSILON = [] 
    
    
       Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 4))

       START = [3, 0]
       GOAL = [3, 7]
    
       actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT]
       q_Lambda()

    plt.legend(loc='best')
    plt.title('Analyzed for 4 different values of $EPSILON$ in Q-Lambda')
    plt.xlabel('Episodes')
    plt.ylabel('Average time steps')
    plt.show()

# King's Moves + Stochastic Wind
The implementation with using the King's Moves plus Stochastic Wind . Re-solve the windy gridworld task assuming eight possible actions, including the diagonal moves, rather than the usual four. Assuming that the effect of the wind, if there is any, is stochastic, sometimes varying by 1 from the mean values given for each column.


In [ ]:
# world height
WORLD_HEIGHT = 7

# world width
WORLD_WIDTH = 10

# probability for exploration
EPSILON = 0.2

# Learning rate or step size
ALPHA = 0.2

# reward
REWARD = -1.0

# GAMMA
GAMMA = 0.75

# Lambda
LAMBDA = 0.5

# wind strength for each column
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

# possible actions
ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3
ACTION_UP_LEFT = 4
ACTION_UP_RIGHT = 5
ACTION_DOWN_RIGHT = 6
ACTION_DOWN_LEFT = 7


Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))

actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT, 
           ACTION_UP_LEFT, ACTION_UP_RIGHT, ACTION_DOWN_RIGHT, ACTION_DOWN_LEFT]


START = [3, 0]
GOAL = [3, 7]

# eligibility trace
e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))

# STOCHASTIC
# stochastic = 1
###############################  Functions  ####################################    

# state transitions
def step(currentState, currentAction):

    i, j = currentState

    # Because of stochastic issue, actions are modified accordingly 
    if WIND[j] != 0:
        # Up
      if currentAction == ACTION_UP:
        return [max(i - 1 - WIND[j], 0), j]
      #Down
      elif currentAction == ACTION_DOWN:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), j]
      # Left
      elif currentAction == ACTION_LEFT:
        return [max(i - WIND[j], 0), max(j - 1, 0)]
      # Right
      elif currentAction == ACTION_RIGHT:
        return [max(i - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
      # Up-Left
      elif currentAction == ACTION_UP_LEFT:
        return [max(i - 1 - WIND[j], 0), max(j - 1, 0)]
      # Up-Right
      elif currentAction == ACTION_UP_RIGHT:
        return [max(i - 1 - WIND[j], 0), min(j + 1, WORLD_WIDTH - 1)]
      # Down-Right
      elif currentAction == ACTION_DOWN_RIGHT:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), min(j + 1, WORLD_WIDTH - 1)]
      # Down-Left
      elif currentAction == ACTION_DOWN_LEFT:
        return [max(min(i + 1 - WIND[j], WORLD_HEIGHT - 1), 0), max(j - 1, 0)]

      else:
        assert False

    else:
    
      wind_strength = WIND[int(currentState[0])] + random.randint(-1, 1)

      # Up
      if currentAction == ACTION_UP:
        return [max(i - 1 - wind_strength, 0), j]
      #Down
      elif currentAction == ACTION_DOWN:
        return [max(min(i + 1 - wind_strength, WORLD_HEIGHT - 1), 0), j]
      # Left
      elif currentAction == ACTION_LEFT:
        return [max(i - wind_strength, 0), max(j - 1, 0)]
      # Right
      elif currentAction == ACTION_RIGHT:
        return [max(i - wind_strength, 0), min(j + 1, WORLD_WIDTH - 1)]
      # Up-Left
      elif currentAction == ACTION_UP_LEFT:
        return [max(i - 1 - wind_strength, 0), max(j - 1, 0)]
      # Up-Right
      elif currentAction == ACTION_UP_RIGHT:
        return [max(i - 1 - wind_strength, 0), min(j + 1, WORLD_WIDTH - 1)]
      # Down-Right
      elif currentAction == ACTION_DOWN_RIGHT:
        return [max(min(i + 1 - wind_strength, WORLD_HEIGHT - 1), 0), min(j + 1, WORLD_WIDTH - 1)]
      # Down-Left
      elif currentAction == ACTION_DOWN_LEFT:
        return [max(min(i + 1 - wind_strength, WORLD_HEIGHT - 1), 0), max(j - 1, 0)]

      else:
        assert False

	


# interaction in a Sarsa episode
def oneEpisode_s(Q):
  
    # track the total time steps in this episode
    time = 0

    # initialize state
    currentState = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  

    # keep going until get to the goal state
    while currentState != GOAL:   

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])
        

        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])
                            
        currentState = newState
        currentAction = newAction

        time += 1

    return time



# interaction in a Sarsa_Lambda episode
def oneEpisode_S_lambda2(Q):

    time = 0

    currentState = START

    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])

    # keep going until get to the goal state
    while currentState != GOAL:
        newState = step(currentState, currentAction)

        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])

        # Sarsa lambda update

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], newAction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    e[i ,j, action] *= LAMBDA * GAMMA

        currentState = newState
        currentAction = newAction

        time += 1

    return time


# interaction in a Q-learning episode
def oneEpisode_q(Q):
    
    time = 0
    
    currentState = START
    
    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        # selecting the new action among all possible action which gives the MAX Q
        newAction = np.argmax(Q[newState[0], newState[1], :])
        
        # Q-learning update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD +  GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])

        currentState = newState

        time += 1

    return time


# interaction in a Watkins’s Q(λ) episode
def oneEpisode_q_Lambda2(Q):

    time = 0
    
    currentState = START

    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        newAction = np.argmax(Q[newState[0], newState[1], :])

        # Watkins’s Q_Lambda update          
        Bestaction = np.argmax(Q[newState[0], newState[1], actions])

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], Bestaction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    if newAction == Bestaction:
                          e[i ,j, action] *= LAMBDA * GAMMA
                    else:
                          e[i ,j, action] == 0      
  
        currentState = newState
        currentAction = newAction

        time += 1

    return time



def sarsa():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    #stochastic == 1
    while ep < episodeLimit:
            #episodes.append(oneEpisode_s(Q))
            time = oneEpisode_s(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='royalblue')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 8 action SARSA, alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')


    
    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 

    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\t    Optimal policy SARSA is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')



def sarsa_Lambda2():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    #stochastic == 1
    while ep < episodeLimit:
            #episodes.append(oneEpisode_S_lambda2(Q))
            time = oneEpisode_S_lambda2(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='red')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 8 action SARSA ($\lambda$), alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')
    plt.show()

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 

    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\tOptimal policy SARSA_lambda is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')


def q_learning():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    while ep < episodeLimit:
            #episodes.append(oneEpisode_q(Q))
            time = oneEpisode_q(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='forestgreen')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 8 action Q-learning, alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')
    plt.show()


    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 

    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\t Optimal policy Q-learning is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')



def q_Lambda():

    Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
    # number of episodes
    episodeLimit = 1000
    ep = 0
    episodes = []
    while ep < episodeLimit:
            #episodes.append(oneEpisode_q_Lambda2(Q))
            time = oneEpisode_q_Lambda2(Q)
            episodes.extend([ep] * time)
            ep += 1

    #episodes = np.add.accumulate(episodes)

    plt.figure(figsize=(8,8))
    plt.plot(episodes, color='purple')
    plt.grid(b=True, which='major', color='#666666', linestyle='--')
    plt.title("Windy Gridworld 8 action Watkins’s Q ($\lambda$), alpha={}, epsilon={}".format(ALPHA, EPSILON))
    plt.xlabel('Time steps')
    plt.ylabel('Episodes')
    plt.show()

    # display the optimal policy
    optimalPolicy = []
    for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 

    print('\n-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-') 
    print('\tOptimal policy Watkins’s Q(λ) is:    \n')         
    for row in optimalPolicy:
            print(row)
    print('\n{}'.format([str(w) for w in WIND]))
    print('\t Wind strength for each column\n')

###############################################################################

if __name__ == '__main__':
    sarsa()
    sarsa_Lambda2()
    q_Lambda()
    q_learning()



# Compare all algorithms in 8 action

In [ ]:

#######################   Comparing   ######################

WORLD_HEIGHT = 7
WORLD_WIDTH = 10
EPSILON = 0.01
ALPHA = 0.1
REWARD = -1.0
GAMMA = 0.9
LAMBDA = 0.8

# wind 
WIND = [0, 0, 0, 1, 1, 1, 2, 2, 1, 0]

ACTION_UP = 0
ACTION_DOWN = 1
ACTION_LEFT = 2
ACTION_RIGHT = 3
ACTION_UP_LEFT = 4
ACTION_UP_RIGHT = 5
ACTION_DOWN_RIGHT = 6
ACTION_DOWN_LEFT = 7

Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))

actions = [ACTION_UP, ACTION_DOWN, ACTION_LEFT, ACTION_RIGHT, 
           ACTION_UP_LEFT, ACTION_UP_RIGHT, ACTION_DOWN_RIGHT, ACTION_DOWN_LEFT]

START = [3, 0]
GOAL = [3, 7]

# eligibility trace
e = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))

###############################  Functions  ####################################    

# state transitions
def step(currentState, currentAction):

    i, j = currentState

    # Because of stochastic issue, actions are modified accordingly
    wind_strength = WIND[int(currentState[0])] + random.randint(-1, 1)

    # Up
    if currentAction == ACTION_UP:
        return [max(i - 1 - wind_strength, 0), j]
    #Down
    elif currentAction == ACTION_DOWN:
        return [max(min(i + 1 - wind_strength, WORLD_HEIGHT - 1), 0), j]
    # Left
    elif currentAction == ACTION_LEFT:
        return [max(i - wind_strength, 0), max(j - 1, 0)]
    # Right
    elif currentAction == ACTION_RIGHT:
        return [max(i - wind_strength, 0), min(j + 1, WORLD_WIDTH - 1)]
    # Up-Left
    elif currentAction == ACTION_UP_LEFT:
        return [max(i - 1 - wind_strength, 0), max(j - 1, 0)]
    # Up-Right
    elif currentAction == ACTION_UP_RIGHT:
        return [max(i - 1 - wind_strength, 0), min(j + 1, WORLD_WIDTH - 1)]
    # Down-Right
    elif currentAction == ACTION_DOWN_RIGHT:
        return [max(min(i + 1 - wind_strength, WORLD_HEIGHT - 1), 0), min(j + 1, WORLD_WIDTH - 1)]
    # Down-Left
    elif currentAction == ACTION_DOWN_LEFT:
        return [max(min(i + 1 - wind_strength, WORLD_HEIGHT - 1), 0), max(j - 1, 0)]

    else:
        assert False



# interaction in a Sarsa episode
def oneEpisode_s(Q):
  
    # track the total time steps in this episode
    time = 0

    # initialize state
    currentState = START

    # choose an action based on epsilon-greedy algorithm
    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  

    # keep going until get to the goal state
    while currentState != GOAL:   

        newState = step(currentState, currentAction)
        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])
        
        # Sarsa update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD + Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])
                            
        currentState = newState
        currentAction = newAction

        time += 1

    return time



# interaction in a Sarsa_Lambda episode
def oneEpisode_S_lambda2(Q):

    time = 0

    currentState = START

    if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
    else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])

    # keep going until get to the goal state
    while currentState != GOAL:
        newState = step(currentState, currentAction)

        if np.random.binomial(1, EPSILON) == 1:
            newAction = np.random.choice(actions)
        else:
            newAction = np.argmax(Q[newState[0], newState[1], :])

        # Sarsa lambda update

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], newAction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    e[i ,j, action] *= LAMBDA * GAMMA

        currentState = newState
        currentAction = newAction

        time += 1

    return time


# interaction in a Q-learning episode
def oneEpisode_q(Q):
    
    time = 0
    
    currentState = START
    
    # keep going until get to the goal state
    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        # selecting the new action among all possible action which gives the MAX Q
        newAction = np.argmax(Q[newState[0], newState[1], :])
        
        # Q-learning update
        Q[currentState[0], currentState[1], currentAction] += \
                            ALPHA * (REWARD +  GAMMA * Q[newState[0], newState[1], newAction] -
                            Q[currentState[0], currentState[1], currentAction])

        currentState = newState

        time += 1

    return time


# interaction in a Watkins’s Q(λ) episode
def oneEpisode_q_Lambda2(Q):

    time = 0
    
    currentState = START

    while currentState != GOAL:
        
        
        if np.random.binomial(1, EPSILON) == 1:
            currentAction = np.random.choice(actions)
        else:
            currentAction = np.argmax(Q[currentState[0], currentState[1], :])  
        newState = step(currentState, currentAction)
        newAction = np.argmax(Q[newState[0], newState[1], :])

        # Watkins’s Q_Lambda update          
        Bestaction = np.argmax(Q[newState[0], newState[1], actions])

        DELTA = REWARD + GAMMA * Q[newState[0], newState[1], Bestaction] - Q[currentState[0], currentState[1], currentAction]
        e[currentState[0], currentState[1], currentAction] += 1
        for i in range(0, WORLD_HEIGHT):
            for j in range(0, WORLD_WIDTH):
                for action in actions:
                    Q[i, j, action] += ALPHA * DELTA * e[i, j, action]
                    if newAction == Bestaction:
                          e[i ,j, action] *= LAMBDA * GAMMA
                    else:
                          e[i ,j, action] == 0      
  
        currentState = newState
        currentAction = newAction

        time += 1

    return time





Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
# number of episodes
episodeLimit = 1000
ep = 0
episodes1 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_s(Q))
            time = oneEpisode_s(Q)
            episodes1.extend([ep] * time)
            ep += 1



    
# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 




Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
# number of episodes
episodeLimit = 1000
ep = 0
episodes2 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_q(Q))
            time = oneEpisode_q(Q)
            episodes2.extend([ep] * time)
            ep += 1




# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 



Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
# number of episodes
episodeLimit = 1000
ep = 0
episodes3 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_S_lambda(Q))
            time = oneEpisode_S_lambda2(Q)
            episodes3.extend([ep] * time)
            ep += 1




# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 




Q = np.zeros((WORLD_HEIGHT, WORLD_WIDTH, 8))
# number of episodes
episodeLimit = 1000
ep = 0
episodes4 = []
while ep < episodeLimit:
            #episodes.append(oneEpisode_q_Lambda(Q))
            time = oneEpisode_q_Lambda2(Q)
            episodes4.extend([ep] * time)
            ep += 1



# display the optimal policy
optimalPolicy = []
for i in range(0, WORLD_HEIGHT):
            optimalPolicy.append([])
            for j in range(0, WORLD_WIDTH):
                if [i, j] == GOAL:
                     optimalPolicy[-1].append('G')
                     continue
                bestAction = np.argmax(Q[i, j, :])
                #print(Q)
                if bestAction == ACTION_UP:
                     optimalPolicy[-1].append('↑')
                elif bestAction == ACTION_DOWN:
                     optimalPolicy[-1].append('↓')
                elif bestAction == ACTION_LEFT:
                     optimalPolicy[-1].append('←')
                elif bestAction == ACTION_RIGHT:
                     optimalPolicy[-1].append('→')
                elif bestAction == ACTION_UP_LEFT:
                     optimalPolicy[-1].append('↖')
                elif bestAction == ACTION_UP_RIGHT:
                     optimalPolicy[-1].append('↗')
                elif bestAction == ACTION_DOWN_RIGHT:
                     optimalPolicy[-1].append('↘')
                elif bestAction == ACTION_DOWN_LEFT:
                     optimalPolicy[-1].append('↙') 



plt.figure(figsize=(10,7))
plt.plot(episodes1, color='darkblue',label="SARSA")
plt.plot(episodes3, color='seagreen',label="SARSA(λ)")
plt.plot(episodes2, color='red',label="Q Learning")
plt.plot(episodes4, color='gold',label="Q(λ)")
plt.grid(b=True, which='major', color='#666666', linestyle='--')
plt.title("All (Windy Gridworld) algorithms - 8 action, alpha={}, epsilon={}".format(ALPHA, EPSILON))
plt.legend(loc='lower right')
plt.xlabel('Time steps')
plt.ylabel('Episodes')
plt.show()

